In [1]:
#imports

import pandas as pd
import numpy as np

First we will gather all the HSL bike data, and make it into a dataframe. 

In [2]:
# Read csv file into a pandas dataframe

bike_data = pd.DataFrame()

for year in range(2016, 2022):
    if year > 2017:
        for month in range(4, 11):
            if month < 10:
                month = '0' + str(month)
            else:
                month = str(month)
        file_name = './data/HSL/od-trips-' + str(year) + '/' + str(year) + '-' + month + '.csv'
        year_bike_data = pd.read_csv(file_name)
        bike_data = pd.concat([bike_data, year_bike_data])
    else:
        for month in range(5, 11):
            if month < 10:
                month = '0' + str(month)
            else:
                month = str(month)
        file_name = './data/HSL/od-trips-' + str(year) + '/' + str(year) + '-' + month + '.csv'
        year_bike_data = pd.read_csv(file_name)
        bike_data = pd.concat([bike_data, year_bike_data])

print(bike_data.head())
print(bike_data.shape)
            
        

             Departure               Return Departure station id  \
0  2016-10-31T23:59:00           2016-11-01                  A24   
1  2016-10-31T23:57:00  2016-10-31T23:58:00                  A23   
2  2016-10-31T23:55:00  2016-10-31T23:57:00                  A22   
3  2016-10-31T23:55:00  2016-10-31T23:58:00                  A38   
4  2016-10-31T23:52:00  2016-10-31T23:55:00                  A21   

  Departure station name Return station id    Return station name  \
0  Rautatientori / länsi               A26                 Kiasma   
1    Rautatientori / itä               A24  Rautatientori / länsi   
2      Kaisaniemenpuisto               A23    Rautatientori / itä   
3      Kauppakorkeakoulu               A30      Kampin metroasema   
4         Varsapuistikko               A22      Kaisaniemenpuisto   

   Covered distance (m)  Duration (sec.)  
0                 222.0             99.0  
1                 179.0             69.0  
2                 273.0             95.0  
3   

Now we have our bike data in a dataframe. The next step is transformation, where we will get rid of data that we do not need.

In [3]:
bike_data = bike_data.drop(['Return station id', 'Departure station id', 'Covered distance (m)', 'Duration (sec.)'], axis=1)

station_data = pd.DataFrame()
station_data['Station name'] = bike_data['Departure station name'].unique()

#Get rid of Relay Box test station
station_data = station_data[:-1]

calendar_data = pd.DataFrame(columns=['Date'])

for year in range(2016, 2022):
    if year == 2016:
        dates = [date for date in (pd.date_range('05/01/' + str(year), end='10/31/' + str(year))) for i in range(len(station_data))]
    elif year == 2017:
        dates = dates + [date for date in (pd.date_range('05/01/' + str(year), end='10/31/' + str(year))) for i in range(len(station_data))]
    else:
        dates = dates + [date for date in (pd.date_range('04/01/' + str(year), end='10/31/' + str(year))) for i in range(len(station_data))]

calendar_data['Date'] = dates

num_dates = len(calendar_data['Date'].unique())

station_data = pd.concat([station_data]*num_dates, ignore_index=True)

calendar_data = pd.concat([calendar_data, station_data], axis=1)

for station in calendar_data['Station name'].unique():
    for date in calendar_data['Date'].unique():
        temp_df = bike_data[(bike_data['Departure station name'] == station) & (str(date) in bike_data['Departure'])]
        departures = len(temp_df)
        temp_df = bike_data[(bike_data['Return station name'] == station) & (str(date) in bike_data['Return'])]
        returns = len(temp_df)
        calendar_data.loc[(calendar_data['Station name'] == station) & (calendar_data['Date'] == date), 'Departures'] = departures
        calendar_data.loc[(calendar_data['Station name'] == station) & (calendar_data['Date'] == date), 'Returns'] = returns

print(calendar_data.head(10))

    
